In [5]:
# Cell 1: Installation of required libraries and OpenAI API Key Setup

%%capture

!pip install langchain==0.1.16
!pip install langchain-core==0.1.37
!pip install langchain-openai==0.1.4
!pip install langchain-community==0.0.21


# Install Hugging Face related libraries for embeddings
!pip install "huggingface==0.0.1"
!pip install "huggingface-hub==0.23.4"
!pip install "sentence-transformers==2.5.1"

# Install vector database
!pip install "chromadb"

# Install utility for downloading files
!pip install "wget==3.2"

# Removed: ibm-watsonx-ai and langchain-ibm as we are switching to OpenAI

print("All required libraries installed successfully!")

In [3]:
import os
from google.colab import userdata

try:
    # IMPORTANT: Ensure 'OPENAI_API_KEY' is the exact name of your secret in Colab
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("OpenAI API key loaded from Colab secrets.")
except Exception as e:
    print(f"Error loading OpenAI API key from Colab secrets: {e}")
    print("Please ensure your 'OPENAI_API_KEY' secret is set in Colab and enabled for this notebook.")
    # You might want to raise an error or exit if the key is critical
    # raise SystemExit("API Key not found. Exiting.")


OpenAI API key loaded from Colab secrets.


In [6]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# LangChain components for document processing
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# OpenAI specific imports for LLM integration
from langchain_openai import ChatOpenAI # Using ChatOpenAI for conversational models

# Utility for downloading files
import wget

In [7]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

file downloaded


In [8]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

In [9]:
loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

16


In [10]:
loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator="\n")
texts = text_splitter.split_documents(documents)
print(len(texts))

18


In [11]:
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

document ingested


In [12]:
parameters = {
    "temperature": 0.5, # Controls randomness. 0.0 for deterministic, higher for more creative.
    "max_tokens": 256,  # Maximum number of tokens to generate in the output.
    # You can add other OpenAI-specific parameters here if needed
}

# Initialize the OpenAI LLM (using ChatOpenAI for conversational tasks)
# 'gpt-3.5-turbo' is a good, cost-effective default. You can try 'gpt-4' for better performance.
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", **parameters)

print(f"OpenAI LLM initialized with model: {openai_llm.model_name}")

OpenAI LLM initialized with model: gpt-3.5-turbo


In [19]:
qa = RetrievalQA.from_chain_type(llm=openai_llm,
                                     chain_type="stuff",
                                     retriever=docsearch.as_retriever(),
                                     return_source_documents=False)

    # Ask the query
query = "what is mobile policy?"
result = qa.invoke(query)
print(result['result']) # Print the answer from the model

The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. It outlines guidelines related to acceptable use, security, confidentiality, cost management, compliance, reporting lost or stolen devices, and consequences for non-compliance. The policy aims to ensure employees use mobile phones in a manner consistent with company values and legal compliance.


In [20]:
# Ask a more high-level question to test summarization
query = "Can you summarize the document for me?"
result = qa.invoke(query)
print(result['result'])

The document provided includes information on the organization's Code of Conduct, Recruitment Policy, and commitment to health and safety. 

The Code of Conduct emphasizes integrity, respect, accountability, safety, and environmental responsibility. It outlines the ethical standards expected from all employees, focusing on honesty, transparency, respect for diversity, accountability for actions, and maintaining a safe and inclusive workplace.

The Recruitment Policy highlights the organization's commitment to equal opportunity, transparency in recruitment processes, and selection based on qualifications and skills. It promotes diversity and inclusion in hiring practices.

The commitment to health and safety is paramount, with a focus on complying with health and safety laws, preventing accidents and injuries, providing training, and fostering open communication on safety concerns.

Overall, the document stresses the importance of ethical conduct, diversity, inclusion, and maintaining a

In [21]:
# Cell 21: Initialize ConversationBufferMemory

memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)
print("Conversation memory initialized.")

Conversation memory initialized.


In [22]:
# Cell 22: Create a ConversationalRetrievalChain with memory

# This chain will use the memory to retain conversation history
qa_with_memory = ConversationalRetrievalChain.from_llm(llm=openai_llm, # Your OpenAI LLM
                                           chain_type="stuff",
                                           retriever=docsearch.as_retriever(),
                                           memory = memory, # Pass the memory object here
                                           get_chat_history=lambda h : h, # Function to format chat history
                                           return_source_documents=False)
print("ConversationalRetrievalChain with memory initialized.")

ConversationalRetrievalChain with memory initialized.


In [23]:
# Cell 23: Create a history list to store chat history (for manual management)
history = []

# Cell 24: First query with memory
query = "What is mobile policy?"
result = qa_with_memory.invoke({"question":query, "chat_history": history})
print(result["answer"])

# Cell 25: Append the previous query and answer to the history
history.append((query, result["answer"]))

# Cell 26: Second query with memory (referencing previous context)
query = "List points in it?" # "it" should refer to mobile policy
result = qa_with_memory.invoke({"question": query, "chat_history": history})
print(result["answer"])

# Cell 27: Append the previous query and answer to the chat history again
history.append((query, result["answer"]))

# Cell 28: Third query with memory (referencing previous context)
query = "What is the aim of it?" # "it" should refer to mobile policy
result = qa_with_memory.invoke({"question": query, "chat_history": history})
print(result["answer"])

A mobile phone policy is a set of guidelines and rules that govern the appropriate and responsible use of mobile devices within an organization. It typically outlines acceptable use, security measures, confidentiality requirements, cost management guidelines, compliance with laws and regulations, reporting procedures for lost or stolen devices, and potential consequences for non-compliance. The policy aims to ensure that employees use mobile phones in a manner consistent with company values, legal requirements, and best practices.
Based on the provided context, a mobile phone policy typically includes the following points:

1. **Cost Management**: Keeping personal phone usage separate from company accounts and reimbursing the company for any personal charges on company-issued phones.
2. **Compliance**: Adhering to all pertinent laws and regulations concerning mobile phone usage, including those related to data protection and privacy.
3. **Lost or Stolen Devices**: Immediately reporting